In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Dense, Conv1D, Dropout, Input, Activation, AveragePooling1D, Flatten
from keras.regularizers import l2
from keras.constraints import MaxNorm
from keras import Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import os
from contextlib import redirect_stdout
from collections import defaultdict
import sys
import seaborn as sns
sys.path.append('../../CommonScripts/')
from utils import checkPath, addBack

Using TensorFlow backend.


In [3]:
model_folder='../Models/easy_split_1/AG-baseline-mod1/'
if os.path.isdir(model_folder):
    raise Exception('Change model name')
else:
    checkPath(model_folder)
train_file='../Seqs_Jin-Standardized/easy-1/train_easy-AG.csv'
test_file='../Seqs_Jin-Standardized/easy-1/test_easy-AG.csv'
seq_len=1200
no_classes=9
no_epochs=50

### Load Data

In [4]:
train_data = pd.read_csv(train_file) 
X_train = train_data.iloc[:, :-1].values
y_train = to_categorical(train_data.iloc[:, -1].values)

In [5]:
test_data = pd.read_csv(test_file)
X_test = test_data.iloc[:, :-1].values
y_test = to_categorical(test_data.iloc[:, -1].values)

In [6]:
X_train = np.reshape(X_train, (-1, seq_len, 1))
X_test = np.reshape(X_test, (-1, seq_len, 1))

### Model Definition

In [8]:
def model_def():
    inputs = Input(shape=(seq_len,1))
    conv1 = Conv1D(filters=48, kernel_size=5, activation='sigmoid', strides=1, kernel_regularizer=l2(l=0.00025), bias_regularizer=l2(l=0.00025))(inputs)
    avg_pool1 = AveragePooling1D(pool_size=2, stride=2)(conv1)
    conv2 = Conv1D(filters=24, kernel_size=5, activation='sigmoid', strides=1, kernel_regularizer=l2(l=0.00025), bias_regularizer=l2(l=0.00025))(avg_pool1)
    avg_pool2 = AveragePooling1D(pool_size=2, stride=2)(conv2)
    flatten = Flatten()(avg_pool2)
    dense1 = Dense(4096, activation='sigmoid', kernel_regularizer=l2(l=0.00025), bias_regularizer=l2(l=0.00025))(flatten)
    dense2 = Dense(512, activation='sigmoid', kernel_regularizer=l2(l=0.00025), bias_regularizer=l2(l=0.00025))(dense1)
    outputs = Dense(no_classes, activation='softmax', kernel_regularizer=l2(l=0.00025), bias_regularizer=l2(l=0.00025))(dense2)
    model = Model(inputs=inputs, outputs=outputs)
    return model

### Train Model

In [ ]:
model = model_def()
print(model.summary())
with open(os.path.join(model_folder + 'structure.txt'), 'w') as f:
    with redirect_stdout(f):
        print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.9), metrics=['accuracy'])
model_checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(model_folder, 'model.hdf5'),
    monitor='val_loss',
    mode='min',
    save_best_only=True)
history = model.fit(X_train, y_train, batch_size=1, epochs=no_epochs, callbacks=[model_checkpoint_callback], validation_data=(X_test, y_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1200, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1196, 48)          288       
_________________________________________________________________
average_pooling1d_1 (Average (None, 598, 48)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 594, 24)           5784      
_________________________________________________________________
average_pooling1d_2 (Average (None, 297, 24)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7128)              0         
___________________________________________

/home/nithya/miniconda3/envs/deepsrgm/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=2, strides=2)`
  after removing the cwd from sys.path.
/home/nithya/miniconda3/envs/deepsrgm/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=2, strides=2)`
  



Train on 5590 samples, validate on 972 samples
Epoch 1/50
3423/5590 [=================>............] - ETA: 6:13 - loss: 3.5467 - accuracy: 0.1177

### Plot Metrics

In [ ]:
def plot_metrics(history):
    fig, axs = plt.subplots(1, 2)
    
    # plot the loss curve
    axs[0].plot(np.arange(no_epochs), history.history['loss'], label='Train')
    axs[0].plot(np.arange(no_epochs), history.history['val_loss'], label='Val')
    axs[0].set(title='Loss Curve', xlabel='Epoch Number', ylabel='Loss', ylim=(0, 10))
    axs[0].legend()
    # plot the accuracy curve
    axs[1].plot(np.arange(no_epochs), history.history['accuracy'], label='Train')
    axs[1].plot(np.arange(no_epochs), history.history['val_accuracy'], label='Val')
    axs[1].set(title='Accuracy Curve', xlabel='Epoch Number', ylabel='Accuracy', ylim=(0, 1))
    axs[1].legend()
    
    fig.savefig(os.path.join(model_folder, 'train.png'))
    
    # save accuracy/loss in a file
    # calculate metrics
    with open(os.path.join(model_folder, 'summary.txt'), 'w') as f:
        f.write('train_loss\ttrain_accuracy\tval_loss\tval_accuracy\tepoch #\n')
        metrics = defaultdict(int)
        min_val_ind = np.argmin(history.history['val_loss'])
        metrics['train_loss'] = history.history['loss'][min_val_ind]
        metrics['train_accuracy'] = history.history['accuracy'][min_val_ind]
        metrics['val_loss'] = history.history['val_loss'][min_val_ind]
        metrics['val_accuracy'] = history.history['val_accuracy'][min_val_ind]
        f.write(f"{metrics['train_loss']}\t{metrics['train_accuracy']}\t{metrics['val_loss']}\t{metrics['val_accuracy']}\t{min_val_ind}/{no_epochs}\n")

In [ ]:
plot_metrics(history)

### Evaluate Model

In [ ]:
raga_labels = ['Bag', 'Bahar', 'Bilas', 'Jaun', 'Kedar', 'MM', 'Marwa', 'Nand', 'Shree']

In [ ]:
model = load_model(os.path.join(model_folder, 'model.hdf5'))
loss, acc = model.evaluate(X_test, y_test)
print(loss, acc)

In [ ]:
predictions = model.predict(X_test)
predict_class = np.argmax(predictions, axis=1)

In [ ]:
sns.heatmap(confusion_matrix(np.argmax(y_test, axis=1), predict_class), xticklabels=raga_labels, yticklabels=raga_labels, annot=True)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(os.path.join(model_folder, 'test_cm.png'))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
sns.heatmap(confusion_matrix(np.argmax(y_train, axis=1), np.argmax(model.predict(X_train), axis=1)), xticklabels=raga_labels, yticklabels=raga_labels, annot=True, fmt="d", ax=ax)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(os.path.join(model_folder, 'train_cm.png'))